In [1]:
import numpy as np
import itertools
from timeit import default_timer as timer

In [2]:
def get_input(name):
    with open(f'{name}.txt') as f:
        return f.read()

# Part 1

In [32]:
class Rock:
    rock_shapes = [ 
        [[3, 3, 3, 3], [0, 1, 2, 3]], # I, lying down
        [[1, 2, 2, 2, 3], [1, 0, 1, 2, 1]], # plus
        [[1, 2, 3, 3, 3], [2, 2, 2, 1, 0]], # backwards L
        [[0, 1, 2, 3], [0, 0, 0, 0]] , # I, standing up
        [[2, 2, 3, 3], [0, 1, 0, 1]]  , # box
          ]
    rock_heights = [1, 3, 3, 4, 2]
    
    def __init__(self, y, rock_shape):
        self.x = 2
        self.y = y
        self.shape = rock_shape
    
    def max_y(self):
        return self.y + HITBOX_SIZE - self.rock_heights[self.shape]
         
    def image(self):
        return zip(*self.rock_shapes[self.shape])

In [33]:
class Cave:
    n_rocks = 0
    state = 'active'
    height_outside_of_view = 0
    
    
    def __init__(self, height, width, rock_limit, jet_streams):
        self.height = height
        self.current_height = height
        self.width = width
        self.field = np.zeros([height+1, width+2])
        # self.field[:, 0] = 1
        # self.field[:, -1] = 1
        # self.field[-1, :] = 1
        self.rock_limit = rock_limit
        self.rock_cycle = itertools.cycle([0, 1, 2, 3, 4])
        self.jet_cycle = itertools.cycle(jet_streams)
          
    def intersects(self):
        intersection = False
        for i,j in self.rock.image():
            if i + self.rock.y > self.height - 1 or \
                    j + self.rock.x > self.width - 1 or \
                    j + self.rock.x < 0 or \
                    self.field[i + self.rock.y][j + self.rock.x] > 0:
                intersection = True
        return intersection
    
#     def intersects(self):
#         intersection = False
#         for i,j in self.rock.image():
#             if self.field[i + self.rock.y][j + self.rock.x]:
#                 intersection = True
#         return intersection    
    
    def new_rock(self):
        self.rock = Rock(self.current_height - HITBOX_SIZE - 3, next(self.rock_cycle))
        
    def freeze(self):
        for i,j in self.rock.image():
            self.field[i + self.rock.y][j + self.rock.x] = 1
        
        self.n_rocks += 1
        self.current_height = min(self.current_height, self.rock.max_y())
        
        if self.current_height < 2*HITBOX_SIZE:
            self.update_field()
            
        if self.n_rocks == self.rock_limit:
            self.state = 'finished'
        else:
            self.new_rock()
            
    def fall_down(self):
        self.rock.y += 1
        if self.intersects():
            self.rock.y -= 1
            self.freeze()

    def go_side(self):
        dx = -1 if next(self.jet_cycle) == '<' else 1
        old_x = self.rock.x
        self.rock.x += dx
        if self.intersects():
            self.rock.x = old_x
    
    
    def get_col_heights(self):
        nz = np.argwhere(self.field)
        lowest_y_for_x = np.zeros(CAVE_WIDTH)
        for x in range(CAVE_WIDTH):
            if x in (cols_w_x := nz[:, 1]):
                lowest_y_for_x[x] = nz[cols_w_x == x][:,0].min()
            else: 
                lowest_y_for_x[x] = self.height
        return lowest_y_for_x
    
    def update_field(self):
        lowest_y_for_x = self.get_col_heights()
        maxy = lowest_y_for_x.max()

        to_roll = int(self.height - maxy)
        self.field = np.roll(self.field, to_roll, axis=0)
        self.field[:to_roll, :] = 0
        
        self.height_outside_of_view += to_roll
        self.current_height += to_roll 
        
    def get_total_height(self):
        return self.height_outside_of_view + self.height - self.current_height

In [34]:
def run_cave_sim(max_height, CAVE_WIDTH, max_rocks, jet_streams):
    c = Cave(max_height, CAVE_WIDTH, max_rocks, jet_streams)
    c.new_rock()
    while not c.state == 'finished':
        c.go_side()
        c.fall_down()
    return c

In [35]:
CAVE_WIDTH = 7
HITBOX_SIZE = 4
rock_limit = 2022
max_height = 1500*HITBOX_SIZE

jet_streams = get_input('test')

c = run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)

In [36]:
c.get_total_height()

3068

In [ ]:
CRT = ''''''
for row in c.field:
    if any(row):
        CRT += '|'
        for pt in row:        
            CRT += '#' if pt else '.'
        CRT += '|\n'
print(CRT)

In [8]:
%load_ext line_profiler

In [37]:
CAVE_WIDTH = 7
HITBOX_SIZE = 4
rock_limit = 100000
max_height = 1500*HITBOX_SIZE

jet_streams = get_input('test')

%lprun -f Cave.intersects run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)

Timer unit: 1e-07 s

Total time: 7.90848 s
File: C:\Users\a336214\AppData\Local\Temp\ipykernel_29020\2398074894.py
Function: intersects at line 19

Line #      Hits         Time  Per Hit   % Time  Line Contents
    19                                               def intersects(self):
    20   1142842    1752891.0      1.5      2.2          intersection = False
    21   4948506   15555189.0      3.1     19.7          for i,j in self.rock.image():
    22   4948502   13135093.0      2.7     16.6              if i + self.rock.y > self.height - 1 or \
    23   4911358   12107860.0      2.5     15.3                      j + self.rock.x > self.width - 1 or \
    24   4828499   10289230.0      2.1     13.0                      j + self.rock.x < 0 or \
    25   4651369   23968424.0      5.2     30.3                      self.field[i + self.rock.y][j + self.rock.x] > 0:
    26    297137     570784.0      1.9      0.7                  intersection = True
    27   1142842    1705281.0      1.5   

In [115]:
CAVE_WIDTH = 7
HITBOX_SIZE = 4
rock_limit = 100000
max_height = 1500*HITBOX_SIZE

jet_streams = get_input('test')

%lprun -f Cave.intersects run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)

Timer unit: 1e-07 s

Total time: 41.7057 s
File: C:\Users\a336214\AppData\Local\Temp\ipykernel_28100\3793717076.py
Function: intersects at line 17

Line #      Hits         Time  Per Hit   % Time  Line Contents
    17                                               def intersects(self):
    18   1142842    3760005.0      3.3      0.9          intersection = False
    19   4571368   19281059.0      4.2      4.6          for i in range(HITBOX_SIZE):
    20  18285472   82973506.0      4.5     19.9              for j in range(HITBOX_SIZE):
    21  13336966  162460631.0     12.2     39.0                  if i * HITBOX_SIZE + j in self.rock.image():
    22   4948502   27550692.0      5.6      6.6                      if i + self.rock.y > self.height - 1 or \
    23   4911358   24135215.0      4.9      5.8                              j + self.rock.x > self.width - 1 or \
    24   4828499   21121232.0      4.4      5.1                              j + self.rock.x < 0 or \
    25   4651369   709

In [18]:
times = []
for rock_limit in [1, 10, 100, 1000, 10000, 100000]:
    if c in locals():
        del c
    t0 = timer()
    CAVE_WIDTH = 7
    HITBOX_SIZE = 4
    # rock_limit = 1000000000000
    max_height = 2500*HITBOX_SIZE

    jet_streams = get_input('test')

    c = run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)
    times.append(timer()-t0)

In [19]:
times

[0.00047600000002034903,
 0.0008498999999915213,
 0.006065000000006648,
 0.0385928999999976,
 0.30672930000000065,
 3.025119999999987]

In [107]:
times

[0.008702499999799329,
 0.0012867000000369444,
 0.00917059999983394,
 0.0826563000000533,
 0.7166383999997379,
 7.264314699999886]

In [ ]:
# 13 years runtime. Probably need to find another way

In [ ]:
CRT = ''''''
for row in c.field:
    if any(row):
        CRT += '|'
        for pt in row:        
            CRT += '#' if pt else '.'
        CRT += '|\n'
print(CRT)

# Part 2